In [1]:
# main.py
import os, math, gc, warnings
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, accuracy_score, confusion_matrix
from sklearn.linear_model import LinearRegression, LogisticRegression
from sklearn.ensemble import (
    RandomForestRegressor, RandomForestClassifier,
    IsolationForest, StackingRegressor, StackingClassifier,
    HistGradientBoostingRegressor, HistGradientBoostingClassifier
)

warnings.filterwarnings("ignore")
np.random.seed(0)

# Optional libs
HAS_XGB = HAS_LGBM = HAS_CAT = HAS_NGB = False
try:
    from xgboost import XGBRegressor, XGBClassifier
    HAS_XGB = True
except Exception:
    pass
try:
    from lightgbm import LGBMRegressor, LGBMClassifier
    HAS_LGBM = True
except Exception:
    pass
try:
    from catboost import CatBoostRegressor, CatBoostClassifier
    HAS_CAT = True
except Exception:
    pass
try:
    from ngboost import NGBRegressor
    from ngboost.distns import Normal
    HAS_NGB = True
except Exception:
    pass

DATA_PATH = "data/AirQualityUCI.csv"
OUTPUT_DIR = "outputs"
FIG_DIR = os.path.join(OUTPUT_DIR, "figures")
os.makedirs(FIG_DIR, exist_ok=True)

TARGETS = ["CO(GT)", "NMHC(GT)", "C6H6(GT)", "NOx(GT)", "NO2(GT)"]
METEO = ["T", "RH", "AH"]
ALL_FEATURES_RAW = TARGETS + METEO
HORIZONS = [1, 6, 12, 24]
LAGS = [1, 6, 12, 24]
MAS = [3, 6, 12, 24]
TRAIN_END = "2004-12-31 23:00"
TEST_END = "2005-02-28 23:00"
USE_CURRENT_LEVELS = True

def rmse(y_true, y_pred):
    return math.sqrt(mean_squared_error(y_true, y_pred))

def load_and_clean(path):
    df = pd.read_csv(path, sep=';', decimal=',', na_values=['NA', 'NaN', ''])
    if df.columns[-1] == "":
        df = df.iloc[:, :-1]
    df.columns = [c.strip() for c in df.columns]
    df["Timestamp"] = pd.to_datetime(df["Date"].astype(str) + " " + df["Time"].astype(str),
                                     dayfirst=True, errors="coerce")
    df = df.dropna(subset=["Timestamp"]).sort_values("Timestamp").reset_index(drop=True)
    num_cols = [c for c in df.columns if c not in ["Date","Time","Timestamp"]]
    for c in num_cols:
        df[c] = pd.to_numeric(df[c], errors="coerce")
        df.loc[df[c] == -200, c] = np.nan
    return df

def impute_base(df, cols):
    df = df.copy()
    df[cols] = df[cols].ffill()
    for c in cols:
        weekday_med = df.groupby(df["Timestamp"].dt.weekday)[c].transform("median")
        df[c] = df[c].fillna(weekday_med)
    df[cols] = df[cols].fillna(df[cols].median())
    return df

def add_time_features(df):
    df = df.copy()
    ts = df["Timestamp"]
    df["hour"] = ts.dt.hour
    df["weekday"] = ts.dt.weekday
    df["month"] = ts.dt.month
    df["is_weekend"] = (df["weekday"] >= 5).astype(int)
    return df

def add_lag_ma_features(df, base_cols, lags, mas):
    df = df.copy()
    for c in base_cols:
        for L in lags:
            df[f"{c}_lag{L}"] = df[c].shift(L)
        for w in mas:
            df[f"{c}_ma{w}"] = df[c].rolling(w, min_periods=max(1, w//2)).mean()
    return df

def temporal_split(df, train_end, test_end):
    train = df[df["Timestamp"] <= train_end].copy()
    test = df[(df["Timestamp"] > train_end) & (df["Timestamp"] <= test_end)].copy()
    return train, test

def build_feature_list(df):
    feats = []
    for c in ALL_FEATURES_RAW:
        if USE_CURRENT_LEVELS:
            feats.append(c)
        for L in LAGS:
            feats.append(f"{c}_lag{L}")
        for w in MAS:
            feats.append(f"{c}_ma{w}")
    feats += ["hour", "weekday", "month", "is_weekend"]
    return [c for c in feats if c in df.columns]

def make_supervised(train, test, target, horizon, feature_cols):
    ytr = train[target].shift(-horizon)
    yte = test[target].shift(-horizon)
    Xtr = train[feature_cols].iloc[:-horizon].copy()
    ytr = ytr.iloc[:-horizon].copy()
    Xte = test[feature_cols].iloc[:-horizon].copy()
    yte = yte.iloc[:-horizon].copy()
    scaler = StandardScaler()
    Xtr = pd.DataFrame(scaler.fit_transform(Xtr), columns=Xtr.columns, index=Xtr.index)
    Xte = pd.DataFrame(scaler.transform(Xte), columns=Xte.columns, index=Xte.index)
    return Xtr, ytr.values.ravel(), Xte, yte.values.ravel()

def discretize_CO(values):
    bins = [-np.inf, 1.5, 2.5, np.inf]
    labels = [0, 1, 2]
    return pd.cut(values, bins=bins, labels=labels).astype(int)

def plot_series(y_true, y_pred, title, path):
    plt.figure(figsize=(10,3.2))
    n = min(500, len(y_true))
    plt.plot(range(n), y_true[:n], label="True")
    plt.plot(range(n), y_pred[:n], label="Pred")
    plt.title(title)
    plt.xlabel("Index")
    plt.ylabel("Value")
    plt.legend()
    plt.tight_layout()
    plt.savefig(path, dpi=160)
    plt.close()

# Load + features
df = load_and_clean(DATA_PATH)
df = impute_base(df, ALL_FEATURES_RAW)
df = add_time_features(df)
df = add_lag_ma_features(df, ALL_FEATURES_RAW, LAGS, MAS)
feature_cols = build_feature_list(df)
train, test = temporal_split(df, TRAIN_END, TEST_END)

# Naive baselines
baseline = []
for target in TARGETS:
    for h in HORIZONS:
        ytrue = test[target].shift(-h).iloc[:-h]
        ypred = test[target].iloc[:-h]
        baseline.append({"target": target, "h": h, "model": "Naive", "rmse": rmse(ytrue.values, ypred.values)})
baseline_df = pd.DataFrame(baseline)
os.makedirs(OUTPUT_DIR, exist_ok=True)
baseline_df.to_csv(os.path.join(OUTPUT_DIR, "results_regression_baseline.csv"), index=False)

# Regression models + ensembling
reg_rows = []
for target in TARGETS:
    for h in HORIZONS:
        Xtr, ytr, Xte, yte = make_supervised(train, test, target, h, feature_cols)
        models = {
            "Linear": LinearRegression(),
            "HGBR": HistGradientBoostingRegressor(max_iter=400, learning_rate=0.06, random_state=0),
            "RF": RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=0),
        }
        if HAS_XGB:
            models["XGB"] = XGBRegressor(
                n_estimators=600, max_depth=8, learning_rate=0.05,
                subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
                random_state=0, tree_method="hist", n_jobs=-1
            )
        if HAS_LGBM:
            models["LGBM"] = LGBMRegressor(
                n_estimators=800, num_leaves=63, learning_rate=0.05,
                subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
                random_state=0
            )
        if HAS_CAT:
            models["CatBoost"] = CatBoostRegressor(
                iterations=800, depth=8, learning_rate=0.05,
                loss_function="RMSE", random_seed=0, verbose=False
            )
        if HAS_NGB:
            models["NGBoost"] = NGBRegressor(Dist=Normal, n_estimators=600, learning_rate=0.05,
                                             verbose=False, random_state=0)

        preds = {}
        for name, mdl in models.items():
            mdl.fit(Xtr, ytr)
            p = mdl.predict(Xte)
            preds[name] = p
            reg_rows.append({"target": target, "h": h, "model": name, "rmse": rmse(yte, p)})

        blend = np.mean(np.column_stack(list(preds.values())), axis=1)
        reg_rows.append({"target": target, "h": h, "model": "BlendMean", "rmse": rmse(yte, blend)})

        base_estimators = []
        for nm in preds.keys():
            if nm == "Linear":
                base_estimators.append((nm, LinearRegression()))
            elif nm == "HGBR":
                base_estimators.append((nm, HistGradientBoostingRegressor(max_iter=400, learning_rate=0.06, random_state=0)))
            elif nm == "RF":
                base_estimators.append((nm, RandomForestRegressor(n_estimators=500, n_jobs=-1, random_state=0)))
            elif nm == "XGB" and HAS_XGB:
                base_estimators.append((nm, XGBRegressor(n_estimators=600, max_depth=8, learning_rate=0.05,
                                                         subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
                                                         random_state=0, tree_method="hist", n_jobs=-1)))
            elif nm == "LGBM" and HAS_LGBM:
                base_estimators.append((nm, LGBMRegressor(n_estimators=800, num_leaves=63, learning_rate=0.05,
                                                          subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
                                                          random_state=0)))
            elif nm == "CatBoost" and HAS_CAT:
                base_estimators.append((nm, CatBoostRegressor(iterations=800, depth=8, learning_rate=0.05,
                                                              loss_function="RMSE", random_seed=0, verbose=False)))
            elif nm == "NGBoost" and HAS_NGB:
                base_estimators.append((nm, NGBRegressor(Dist=Normal, n_estimators=600, learning_rate=0.05,
                                                         verbose=False, random_state=0)))
        if len(base_estimators) >= 2:
            stack = StackingRegressor(estimators=base_estimators, final_estimator=LinearRegression())
            stack.fit(Xtr, ytr)
            sp = stack.predict(Xte)
            reg_rows.append({"target": target, "h": h, "model": "StackingLR", "rmse": rmse(yte, sp)})

            best_row = min([r for r in reg_rows if r["target"]==target and r["h"]==h and r["model"] in models],
                           key=lambda x: x["rmse"])
            best_pred = preds[best_row["model"]]
            plot_series(yte, best_pred, f"{target} +{h}h Best:{best_row['model']}",
                        os.path.join(FIG_DIR, f"{target}_h{h}_best.png"))
            plot_series(yte, blend, f"{target} +{h}h BlendMean",
                        os.path.join(FIG_DIR, f"{target}_h{h}_blend.png"))
            plot_series(yte, sp, f"{target} +{h}h Stacking",
                        os.path.join(FIG_DIR, f"{target}_h{h}_stack.png"))
        else:
            best_row = min([r for r in reg_rows if r["target"]==target and r["h"]==h and r["model"] in models],
                           key=lambda x: x["rmse"])
            best_pred = preds[best_row["model"]]
            plot_series(yte, best_pred, f"{target} +{h}h Best:{best_row['model']}",
                        os.path.join(FIG_DIR, f"{target}_h{h}_best.png"))
            plot_series(yte, blend, f"{target} +{h}h BlendMean",
                        os.path.join(FIG_DIR, f"{target}_h{h}_blend.png"))
        gc.collect()

reg_df = pd.DataFrame(reg_rows + baseline_df.to_dict("records"))
reg_df.to_csv(os.path.join(OUTPUT_DIR, "results_regression.csv"), index=False)

# Robust training via anomaly filtering (example CO +1h)
Xtr, ytr, Xte, yte = make_supervised(train, test, "CO(GT)", 1, feature_cols)
base = HistGradientBoostingRegressor(max_iter=400, learning_rate=0.06, random_state=0).fit(Xtr, ytr)
resid = ytr - base.predict(Xtr)
tau = np.quantile(np.abs(resid), 0.95)
mask = (np.abs(resid) <= tau)
iso = IsolationForest(n_estimators=300, contamination=0.05, random_state=0).fit_predict(np.c_[Xtr, ytr]) == 1
robust_mask = mask & iso
if robust_mask.mean() < 0.7:
    robust_mask = mask
robust = HistGradientBoostingRegressor(max_iter=400, learning_rate=0.06, random_state=0).fit(Xtr[robust_mask], ytr[robust_mask])
rmse_base = rmse(yte, base.predict(Xte))
rmse_rob = rmse(yte, robust.predict(Xte))
plt.figure(figsize=(6,3))
plt.hist(np.clip(resid, -5, 5), bins=60)
plt.title("CO +1h Train Residuals (clipped)")
plt.tight_layout()
plt.savefig(os.path.join(FIG_DIR, "CO_h1_residual_hist.png"), dpi=160)
plt.close()

# CO level classification
cls_rows = []
for h in HORIZONS:
    ytr_cls = discretize_CO(train["CO(GT)"].shift(-h)).iloc[:-h]
    yte_cls = discretize_CO(test["CO(GT)"].shift(-h)).iloc[:-h]
    Xtr, _, Xte, _ = make_supervised(train, test, "CO(GT)", h, feature_cols)
    mask_valid = (~pd.isna(ytr_cls)).values
    Xtr_c = Xtr.iloc[mask_valid]
    ytr_c = ytr_cls.values[mask_valid]

    clfs = {
        "LogReg": LogisticRegression(max_iter=2000, multi_class="multinomial", random_state=0),
        "HGBC": HistGradientBoostingClassifier(max_iter=400, learning_rate=0.08, random_state=0),
        "RF": RandomForestClassifier(n_estimators=500, n_jobs=-1, random_state=0),
    }
    if HAS_XGB:
        clfs["XGBc"] = XGBClassifier(
            n_estimators=600, max_depth=8, learning_rate=0.06,
            subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
            random_state=0, tree_method="hist", n_jobs=-1
        )
    if HAS_LGBM:
        clfs["LGBMc"] = LGBMClassifier(
            n_estimators=800, num_leaves=63, learning_rate=0.06,
            subsample=0.8, colsample_bytree=0.8, reg_lambda=1.0,
            random_state=0
        )
    if HAS_CAT:
        clfs["CatBoostc"] = CatBoostClassifier(
            iterations=800, depth=8, learning_rate=0.06,
            loss_function="MultiClass", random_seed=0, verbose=False
        )

    preds = []
    for name, clf in clfs.items():
        clf.fit(Xtr_c, ytr_c)
        yp = clf.predict(Xte)
        acc = accuracy_score(yte_cls, yp)
        cls_rows.append({"h": h, "model": name, "accuracy": acc})
        preds.append(yp)

    pred_arr = np.column_stack(preds)
    vote = []
    for i in range(pred_arr.shape[0]):
        vals, cnts = np.unique(pred_arr[i], return_counts=True)
        vote.append(vals[np.argmax(cnts)])
    vote = np.array(vote)
    cls_rows.append({"h": h, "model": "VoteEnsemble", "accuracy": accuracy_score(yte_cls, vote)})

    cm = confusion_matrix(yte_cls, vote, labels=[0,1,2])
    fig = plt.figure(figsize=(4,3))
    plt.imshow(cm, interpolation='nearest')
    plt.title(f"CO class +{h}h — Vote")
    plt.xticks([0,1,2], ["Low","Mid","High"])
    plt.yticks([0,1,2], ["Low","Mid","High"])
    for (ii, jj), v in np.ndenumerate(cm):
        plt.text(jj, ii, str(v), ha='center', va='center')
    plt.tight_layout()
    fig.savefig(os.path.join(FIG_DIR, f"CO_cls_h{h}_cm.png"), dpi=160)
    plt.close(fig)

cls_df = pd.DataFrame(cls_rows)
cls_df.to_csv(os.path.join(OUTPUT_DIR, "results_classification.csv"), index=False)

print("Artifacts:")
print(" - outputs/results_regression.csv")
print(" - outputs/results_regression_baseline.csv")
print(" - outputs/results_classification.csv")
print(" - outputs/figures/*")
print(f"Robustness (CO +1h): base RMSE={rmse_base:.4f}, robust RMSE={rmse_rob:.4f}")


ValueError: Found array with 0 sample(s) (shape=(0,)) while a minimum of 1 is required.